In [ ]:
# STEP 1: Install dependencies (Demucs + Essentia + FFmpeg binary)
!pip -q install demucs essentia ffmpeg-python
!apt-get -qq update && apt-get -qq install -y ffmpeg

# STEP 2: Upload audio file
from google.colab import files as colab_files
import os, shutil, subprocess, glob

print("🎵 Upload an audio file (MP3, WAV, etc.)")
uploaded = colab_files.upload()
input_filename = list(uploaded.keys())[0]

# Optional: clean up any previous runs
shutil.rmtree("separated", ignore_errors=True)

# STEP 3: Split stems using Demucs
# Try a 6-stem model first; if unavailable, fall back to 4-stem
preferred_models = ["htdemucs_6s", "htdemucs_ft", "hdemucs_mmi", "htdemucs"]
model_used = None
for m in preferred_models:
    print(f"🎚️ Trying Demucs model: {m} ...")
    result = subprocess.run(
        ["demucs", "-n", m, "--out", "separated", input_filename],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    if result.returncode == 0:
        model_used = m
        print(f"✅ Demucs finished with model: {m}")
        break
    else:
        print(f"⚠️ Model {m} failed, trying next...\n{result.stdout[:600]}...")

if not model_used:
    raise RuntimeError("Demucs failed for all models tried. Check the console output above.")

# STEP 4: Analyze key and BPM using Essentia
print("🔍 Detecting key and BPM...")
import essentia.standard as es

audio = es.MonoLoader(filename=input_filename)()
bpm, _, _, _, _ = es.RhythmExtractor2013()(audio)
key, scale, strength = es.KeyExtractor()(audio)

print(f"🎼 Estimated Key: {key} {scale} (strength {strength:.2f})")
print(f"🎵 Estimated BPM: {round(bpm)}")

# STEP 5: Find Demucs output directory robustly and ZIP it
# Demucs output pattern: separated/<model_used>/<track_name>/*
track_name = os.path.splitext(os.path.basename(input_filename))[0]
candidate_dir = os.path.join("separated", model_used, track_name)

# In rare cases, Demucs may sanitize the folder name; fall back to glob if needed
if not os.path.isdir(candidate_dir):
    matches = glob.glob(os.path.join("separated", model_used, "*"))
    # choose the most recent folder
    matches = [d for d in matches if os.path.isdir(d)]
    if not matches:
        raise FileNotFoundError("Could not find Demucs output folder.")
    candidate_dir = max(matches, key=os.path.getmtime)

# Sanity check: ensure stems exist
stem_files = sorted(glob.glob(os.path.join(candidate_dir, "*.*")))
if not stem_files:
    raise FileNotFoundError(f"No stems found in {candidate_dir}. Check Demucs output above.")

# Make a clean zip archive (avoid name collisions with 'files')
base_name = f"{track_name}_{'6stems' if '6' in model_used else '4stems'}"
archive_path = shutil.make_archive(base_name, 'zip', candidate_dir)

print(f"📦 Created archive: {archive_path} (model used: {model_used})")
colab_files.download(archive_path)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 3.4 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?

Saving Another Light.wav to Another Light.wav
🎚️ Trying Demucs model: htdemucs_6s ...
✅ Demucs finished with model: htdemucs_6s
🔍 Detecting key and BPM...
🎼 Estimated Key: G minor (strength 0.80)
🎵 Estimated BPM: 140
📦 Created archive: /content/Another Light_6stems.zip (model used: htdemucs_6s)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>